**Import libraries and data**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#supress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os

directories = ['../input/100-bird-species/birds_rev2/test',
                                '../input/100-bird-species/birds_rev2/train',
                                '../input/100-bird-species/birds_rev2/valid']
for dir in directories:
    label = []
    path = []
    for dirname, _,filenames in os.walk(dir):
        for filename in filenames:
            label.append(os.path.split(dirname)[1])
            path.append(os.path.join(dirname,filename))
    if dir == directories[0]:
        df_test = pd.DataFrame(columns=['path','label'])
        df_test['path']=path
        df_test['label']=label
    elif dir == directories[1]:
        df_train = pd.DataFrame(columns=['path','label'])
        df_train['path']=path
        df_train['label']=label        
    elif dir == directories[2]:
        df_valid = pd.DataFrame(columns=['path','label'])
        df_valid['path']=path
        df_valid['label']=label

# EDA

In [ ]:
df_train.head()

In [ ]:
# Display 20 picture of the dataset with their labels
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

df_sample = df_train.sample(15)
df_sample.reset_index(drop=True, inplace=True)

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_sample.path[i]))
    ax.set_title(df_sample.label[i])
plt.tight_layout()
plt.show()

# Neural Network with Transfer Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_size = (224,224)
image_shape=(224,224,3)

**Baseline**

In [ ]:
BS = 32

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = df_train, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe = df_valid, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

test_generator = valid_datagen.flow_from_dataframe(
    dataframe = df_test, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False)

for _ in range(3):
    img, label = train_generator.next()
    plt.imshow(img[0])
    plt.show()

In [ ]:
#model

resnet_model=tf.keras.applications.InceptionResNetV2(input_shape=image_shape,
                                               include_top=False,
                                               weights='imagenet')
resnet_model.trainable = False


baseline_model = keras.Sequential([  
    resnet_model,
    layers.Flatten(),
    layers.Dense(units=275, activation="softmax"),
])
baseline_model.summary()

baseline_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#fit

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=10)

history = baseline_model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=25,
    verbose=1,
    callbacks=[early_stop]
)

result=pd.DataFrame(history.history)
fig, ax=plt.subplots(nrows=1, ncols=2,figsize=(18,6))
ax=ax.flatten()
ax[0].plot(result[['accuracy','val_accuracy']])
ax[0].set_title("Accuracy")
ax[1].plot(result[['loss','val_loss']])
ax[1].set_title("Loss")

In [ ]:
baseline_model.evaluate(test_generator,use_multiprocessing=True,workers=10)

**Decreasing overfitting with image augmentation**

In [ ]:
BS = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=5,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   shear_range=0.05,
                                   zoom_range=0.05,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = df_train, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe = df_valid, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

test_generator = valid_datagen.flow_from_dataframe(
    dataframe = df_test, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

for _ in range(3):
    img, label = train_generator.next()
    plt.imshow(img[0])
    plt.show()

In [ ]:
#model

resnet_model=tf.keras.applications.InceptionResNetV2(input_shape=image_shape,
                                               include_top=False,
                                               weights='imagenet')
resnet_model.trainable = False


aug_model = keras.Sequential([  
    resnet_model,
    layers.Flatten(),
    layers.Dense(units=275, activation="softmax"),
])
aug_model.summary()

aug_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#fit

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=10)

history = aug_model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=25,
    verbose=1,
    callbacks=[early_stop]
)

result=pd.DataFrame(history.history)
fig, ax=plt.subplots(nrows=1, ncols=2,figsize=(18,6))
ax=ax.flatten()
ax[0].plot(result[['accuracy','val_accuracy']])
ax[0].set_title("Accuracy")
ax[1].plot(result[['loss','val_loss']])
ax[1].set_title("Loss")

In [ ]:
aug_model.evaluate(test_generator,use_multiprocessing=True,workers=10)

**Improving the neural network with additional layers**

In [ ]:
BS = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=5,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   shear_range=0.05,
                                   zoom_range=0.05,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = df_train, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe = df_valid, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

test_generator = valid_datagen.flow_from_dataframe(
    dataframe = df_test, x_col='path', y_col='label',
    target_size= image_size,
    batch_size=BS,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True)

for _ in range(3):
    img, label = train_generator.next()
    plt.imshow(img[0])
    plt.show()

In [ ]:
#model

resnet_model=tf.keras.applications.InceptionResNetV2(input_shape=image_shape,
                                               include_top=False,
                                               weights='imagenet')
resnet_model.trainable = False


layers_model = keras.Sequential([  
    resnet_model,
    layers.Flatten(),
    layers.Dense(units=1950,activation='relu'),
    layers.BatchNormalization(),
#     layers.Dropout(.3),
    layers.Dense(units=275, activation="softmax"),
])
layers_model.summary()

layers_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#fit

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=10)

history = layers_model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=25,
    verbose=1,
    callbacks=[early_stop]
)

result=pd.DataFrame(history.history)
fig, ax=plt.subplots(nrows=1, ncols=2,figsize=(18,6))
ax=ax.flatten()
ax[0].plot(result[['accuracy','val_accuracy']])
ax[0].set_title("Accuracy")
ax[1].plot(result[['loss','val_loss']])
ax[1].set_title("Loss")

In [ ]:
layers_model.evaluate(test_generator,use_multiprocessing=True,workers=10)